In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("lab.ipynb")

# Lab 3 – Merging and Pivoting

## DSC 80, Spring 2025

### Due Date: Wednesday, April 23rd at 11:59PM

## Instructions

Welcome to the third DSC 80 lab this quarter!

Much like in DSC 10, this Jupyter Notebook contains the statements of the problems and provides code and Markdown cells to display your answers to the problems. Unlike DSC 10, the notebook is *only* for displaying a readable version of your final answers. The coding will be done in an accompanying `lab.py` file that is imported into the current notebook, and **you will only submit that `lab.py` file**, not this notebook!

Some additional guidelines:
- **Unlike in DSC 10, labs will have both public tests and hidden tests.** The bulk of your grade will come from your scores on hidden tests, which you will only see on Gradescope after the assignment deadline.
- **Do not change the function names in the `lab.py` file!** The functions in the `lab.py` file are how your assignment is graded, and they are graded by their name. If you changed something you weren't supposed to, you can find the original code in the [course GitHub repository](https://github.com/dsc-courses/dsc80-2025-wi).
- Notebooks are nice for testing and experimenting with different implementations before designing your function in your `lab.py` file. You can write code here, but make sure that all of your real work is in the `lab.py` file, since that's all you're submitting.
- You are encouraged to write your own additional helper functions to solve the lab, as long as they also end up in `lab.py`.

**To ensure that all of the work you want to submit is in `lab.py`, we've included a script named `lab-validation.py` in the lab folder. You shouldn't edit it, but instead, you should call it from the command line (e.g. the Terminal) to test your work.** More details on its usage are given at the bottom of this notebook.

**Importing code from `lab.py`**:

* Below, we import the `.py` file that's contained in the same directory as this notebook.
* We use the `autoreload` notebook extension to make changes to our `lab.py` file immediately available in our notebook. Without this extension, we would need to restart the notebook kernel to see any changes to `lab.py` in the notebook.
    - `autoreload` is necessary because, upon import, `lab.py` is compiled to bytecode (in the directory `__pycache__`). Subsequent imports of `lab` merely import the existing compiled python.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from lab import *

In [4]:
import os
import io
from pathlib import Path
import pandas as pd
import numpy as np

<div class="alert alert-block alert-danger" markdown="1">

**There are only two functions in this lab in which you may use a `for`-loop:**
- **`read_linkedin_survey` in Question 1.**
- **`read_student_surveys` in Question 2.**
    
**You may lose points if you use a `for`-loop or `while`-loop in any other question!**

</div>

## Part 1: Combining Data

### Question 1 – Making Connections 🤝

A group of students decided to send out a survey to their connections on LinkedIn. Each student asks 1000 of their connections for their first and last name, the company they currently work at, their job title, their email, and the university they attended.

**Your job is to combine all the data contained in the files `survey*.csv` (stored within the `data/responses` folder) into a single DataFrame. The number of files and the number of rows in each file may vary, so don't hardcode your answers!** To do so, implement the following two functions.

#### `read_linkedin_survey`

Complete the implementation of the function `read_linkedin_survey`, which takes in a Path for the directory where `survey*.csv` files are located and outputs a single DataFrame with six columns titled `'first name'`, `'last name'`, `'current company'`, `'job title'`, `'email'`, and `'university'` (in that order) containing the all of survey information from all the `.csv` files. Make sure to reset the index of the combined DataFrame before returning it so that the index is unique. When the function takes in an invalide directory it should raise a `FileNotFoundError`.

***Hints***:

- Take a look at a few of the files in the `responses` folder. You may have to do some data cleaning to combine the DataFrames!

- You can list the files in a directory using `.iterdir()`.

***Note***: Remember to use `Path()` to build file paths. Calling `.iterdir()` on a `Path()` object will automatically throw a `FileNotFoundError`.

<br>

#### `com_stats`

Complete the implementation of the function `com_stats`, which takes in a DataFrame returned by `read_linkedin_survey` and returns a list containing, in the following order: 
- The proportion of people who went to a university with the string `'Ohio'` in its name that have the string `'Programmer'` somewhere in their job title.
- The number of job titles that **end** with the exact string `'Engineer'`. Note that we're asking for the number of job titles, **not** the number of people!
- The job title that has the longest name (there are no ties).
- The number of people who have the word `'manager'` in their job title, uppercase or lowercase (`'Manager'`, `'manager'`, and `'mANAgeR'` should all count).

In [ ]:
connects_fp = Path('data') / 'responses'

# open file path to create a file object 
names_fp = Path(connects_fp)
surveys_cvs = []

try:
    for file in names_fp.iterdir():
        surveys_cvs.append(file)
except FileNotFoundError:
    raise

dfs_ls = []
for file in surveys_cvs:
    new_df = pd.read_csv(file)
    dfs_ls.append(new_df)

ordered_col = ['first name', 'last name', 'current company', 'job title', 'email', 'university']

combined_df = pd.DataFrame(columns=ordered_col)
for df in dfs_ls:
    df.columns = df.columns.str.replace('_',' ').str.lower()
    combined_df = pd.concat([combined_df,df],ignore_index=True)

# combined_df

In [ ]:
ohio_filtered_df = combined_df[combined_df['university'].str.contains('Ohio',na=False)]
prop_ohio_programmer = ohio_filtered_df[ohio_filtered_df['job title'].str.contains('Programmer',na=False)].shape[0] / ohio_filtered_df.shape[0]
prop_ohio_programmer

# -----------------------------------------------------------------------------------------------------------------------------
num_eng_jobs = len(combined_df[combined_df['job title'].str.endswith('Engineer',na=False)]['job title'].unique())
# -----------------------------------------------------------------------------------------------------------------------------
longest_job_name_df = combined_df.copy().dropna()
max_index = longest_job_name_df['job title'].str.len().idxmax()
longest_job_title = combined_df.iloc[max_index]['job title']
# -----------------------------------------------------------------------------------------------------------------------------
find_manager_df = combined_df.copy()
find_manager_df['job title'] = find_manager_df['job title'].str.lower()
num_managers = find_manager_df['job title'].str.contains('manager').sum()

369

In [299]:
# do not edit this cell -- it is needed for the tests
dirname = Path('data') / 'responses'
q1_out = read_linkedin_survey(dirname)
stats_out = com_stats(q1_out)

In [300]:
grader.check("q1")

q1 results: All test cases passed!

### Question 2 – Survey Says... 👨‍👩‍👧‍👦

Professor Aritra often sends out extra credit surveys asking students for their favorite animals, movies, and other favorite things. These surveys are stored in the `data/extra-credit-surveys` folder. Each file in that folder corresponds to a different survey question (except for `favorite1.csv`, which contains students' names and IDs).

Here's how extra credit works:
- Each student who has completed at least 50% of the survey questions receives 5 points of extra credit.
- If there is at least one survey question that at least 90% of the class answered (e.g. favorite animal), **everyone** in the class receives 1 point of extra credit. This overall class extra credit only applies twice, so if for example 95% of students answer the favorite color survey question and 91% answer the favorite animal survey question, and and 97% answer the favorite movie question, the entire class still receives 2 extra point as a class, not 3.
- Note that this means that the most extra credit any student can earn is 7 points.

#### `read_student_surveys`

Complete the implementation of the function `read_student_surveys` which takes in a Path describing the directory to a folder containing `favorite*.csv` files and outputs a DataFrame containing all of the survey data combined, indexed by student ID (a value 1-1000).

***Note***: Remember to use `Path()` to build file paths.


<br>

#### `check_credit`

Complete the implementation of the function `check_credit` which takes in a DataFrame returned by `read_student_surveys` and outputs a DataFrame indexed by student ID (a value 1-1000) with two columns:
- `'name'`, containing the name of each student, and
- `'ec'`, containing the number of extra credit points each student earned.

***Note***: For the genres survey, `'(no genres listed)'` does not count as a valid response for receiving extra credit. 

In [301]:
dirname = Path('data') / 'extra-credit-surveys'
file_fp = Path(dirname)

all_questions = []
for file in file_fp.iterdir():
    all_questions.append(file)

questions_dfs_ls = []
for file in all_questions:
    new_df = pd.read_csv(file)
    questions_dfs_ls.append(new_df)

merged_questions_df = pd.DataFrame({'id':np.arange(1,1001)})
for df in questions_dfs_ls:
    merged_questions_df = merged_questions_df.merge(df, on='id')

merged_questions_df = merged_questions_df.set_index('id')
#merged_questions_df

In [280]:
# clean col values 
invalid_values = ["-", "Unknown", "", "None", "N/A", "(no genres listed)"]
merged_questions_df = merged_questions_df.replace(invalid_values,np.nan)

In [296]:
# (num of null entries for each question) / (total num of entries)
# check wherever this is < 0.10 because we want at least 90% of students to answer so for there to be 10% null values allowed 
num_entire_class_earns = ((merged_questions_df.set_index('name').isna().sum() / merged_questions_df.shape[0]) < 0.10).sum()

if num_entire_class_earns > 2: 
    num_entire_class_earns = 2 

swapped_df = merged_questions_df.copy()
swapped_df = swapped_df.set_index('name')
num_questions = len(swapped_df.columns)

# (num of null entries per student) / (num of total questions) == prop of null entries per student 
# want this to be less than 50%
students_plus_five = (swapped_df.T.isna().sum() / num_questions) < 0.5

swapped_df['recieves 5 EC'] = (students_plus_five.values) * 5 
swapped_df['total ec'] = swapped_df['recieves 5 EC'].values + num_entire_class_earns

ec_per_student = pd.DataFrame()
ec_per_student.index = merged_questions_df.index
ec_per_student['name'] = swapped_df.index
ec_per_student['ec'] = swapped_df['total ec'].values

In [297]:
# do not edit this cell -- it is needed for the tests
dirname = Path('data') / 'extra-credit-surveys'
q2_out = read_student_surveys(dirname)
check_credit_out = check_credit(q2_out)

In [298]:
grader.check("q2")

q2 results: All test cases passed!

### Question 3 – Paw Patrol 🐾

You are analyzing data from a veterinarian clinic. The datasets contain several types of information from the clinic, including its customers (pet owners), pets, available procedures, and procedure history. The column names are self-explanatory. These DataFrames are provided to you:
-  `owners` stores the customer information, where every `'OwnerID'` is unique (verify this yourself).
-  `pets` stores the pet information. Each pet belongs to a customer in `owners`.
-  `procedure_detail` contains a catalog of procedures that are offered by the clinic.
-  `procedure_history` has procedure records. Most procedures were given to a pet in `pets`.

Complete the implementation of the following three functions, which each ask you to answer a specific question.

#### `most_popular_procedure`

What is the most popular `'ProcedureType'` amongst all pets in the `pets` DataFrame? Complete the implementation of the function `most_popular_procedure`, which takes in two DataFrames, `pets` and `procedure_history`, and returns the name of the most popular `'ProcedureType'` as a string.

Note that some pets are registered but haven't had any procedures performed. Also, some pets that have had procedures done are not registered in `pets`.

<br>

#### `pet_name_by_owner`

What is the name of each customer's pet(s)? Complete the implementation of the function `pet_name_by_owner`, which takes in two DataFrames, `owners` and `pets`, and returns a Series whose index contains owner first names, and whose values are pet names as **strings**. If an owner has multiple pets, the value corresponding to that owner should instead be a **list of pet names as strings**.

Note that owner first names are not necessarily unique, and so the Series you return will not necessarily have a unique index.

<br>

#### `total_cost_per_city`

Note that the `owners` DataFrame has a `'City'` column, describing the city in which each pet owner and their pets live. How much did each city spend in total on procedures? Complete the implementation of the function `total_cost_per_city`, which takes in four DataFrames, `owners`, `pets`, `procedure_history`, and `procedure_detail`, and returns a Series indexed by `'City'` that describes the total amount that each city has spent on pets' procedures.

***Hint***: At some point, you may have to merge on multiple columns.

***Note***: Some owners may have never visited the veterinarian clinic in their city. This means some cities may have zero operational costs.

In [ ]:
pets_fp = Path('data') / 'pets' / 'Pets.csv'
procedure_history_fp =  Path('data') / 'pets' / 'ProceduresHistory.csv'
owners_fp = Path('data') / 'pets' / 'Owners.csv'
procedure_detail_fp = Path('data') / 'pets' / 'ProceduresDetails.csv'
pets = pd.read_csv(pets_fp)
procedure_history = pd.read_csv(procedure_history_fp)
owners = pd.read_csv(owners_fp)
procedure_detail = pd.read_csv(procedure_detail_fp)

# --------------------------------------- most_popular_procedure -----------------------------------
pet_procedure_df = pets.merge(procedure_history, on='PetID',how='left')
most_popular_treatment = pet_procedure_df.groupby('ProcedureType').count().sort_values('PetID',ascending=False).index[0]
# --------------------------------------- pet_name_by_owner ----------------------------------------
owners_and_pets = owners.merge(pets,on='OwnerID')     # owner name == 'Name_x', pet name == 'Name_y'
owner_names = owners_and_pets.groupby('OwnerID')['Name_x'].agg(list).apply(lambda lst: lst[0])
pet_names = owners_and_pets.groupby('OwnerID')['Name_y'].agg(list)

pets_per_owner = pd.DataFrame({'PetName':pet_names.values},index=owner_names.values)
pets_per_owner['PetName']
# -------------------------------------- total_cost_per_city --------------------------------------
# owners --> City
# pets --> owners by OwnerID
# procedure_history --> pets by PetID
# procedure_details --> procedure_history by ['ProcedureType','ProcedureSubCode']

# want to keep all the procedures performed --> all values in procedure_history
all_procedure_costs = procedure_history.merge(procedure_detail,on=['ProcedureType','ProcedureSubCode'],how='left')
pets_cost = all_procedure_costs.merge(pets,on='PetID')
cost_and_owner = pets_cost.merge(owners,on='OwnerID')
cost_per_city = cost_and_owner.groupby('City')['Price'].sum()
owners
# cities with 0 cost are not found in output 



,OwnerID,Name,Surname,StreetAddress,City,State,StateFull,ZipCode
0,6049,Debbie,Metivier,315 Goff Avenue,Grand Rapids,MI,Michigan,49503
1,2863,John,Sebastian,3221 Perry Street,Davison,MI,Michigan,48423
2,3518,Connie,Pauley,1539 Cunningham Court,Bloomfield Township,MI,Michigan,48302
3,3663,Lena,Haliburton,4217 Twin Oaks Drive,Traverse City,MI,Michigan,49684
4,1070,Jessica,Velazquez,3861 Woodbridge Lane,Southfield,MI,Michigan,48034
...,...,...,...,...,...,...,...,...
84,2103,Robert,Adkins,2102 Perry Street,Flint,MI,Michigan,48548
85,4464,Daniel,Nielson,4876 Tully Street,Detroit,MI,Michigan,48219
86,5737,Alden,McMiller,3111 Tennessee Avenue,Pontiac,MI,Michigan,48342
87,9850,Gary,Snider,3139 Nash Street,Detroit,MI,Michigan,48227


In [429]:
procedure_history

,PetID,Date,ProcedureType,ProcedureSubCode
0,A8-1181,2016-01-10,VACCINATIONS,5
1,E7-3766,2016-01-11,VACCINATIONS,5
2,B8-8740,2016-01-11,VACCINATIONS,5
3,D4-9443,2016-01-11,VACCINATIONS,5
4,F6-3398,2016-01-12,HOSPITALIZATION,1
...,...,...,...,...
2279,A4-2573,2016-12-30,VACCINATIONS,5
2280,B3-8317,2016-12-30,VACCINATIONS,5
2281,C5-4291,2016-12-30,ORTHOPEDIC,3
2282,C6-3299,2016-12-31,VACCINATIONS,5


In [ ]:
# do not edit this cell -- it is needed for the tests
pets_fp = Path('data') / 'pets' / 'Pets.csv'
procedure_history_fp =  Path('data') / 'pets' / 'ProceduresHistory.csv'
owners_fp = Path('data') / 'pets' / 'Owners.csv'
procedure_detail_fp = Path('data') / 'pets' / 'ProceduresDetails.csv'
pets = pd.read_csv(pets_fp)
procedure_history = pd.read_csv(procedure_history_fp)
owners = pd.read_csv(owners_fp)
procedure_detail = pd.read_csv(procedure_detail_fp)

out_01 = most_popular_procedure(pets, procedure_history)
out_02 = pet_name_by_owner(owners, pets)
out_03 = total_cost_per_city(owners, pets, procedure_history, procedure_detail)

In [ ]:
grader.check("q3")

## Part 2: Pivot Tables

### Question 4 – Summarizing Sales 💰

Recall from [Lecture 3](https://dsc80.com/resources/lectures/lec03/lec03-filled.html#Pivot-tables-using-the-pivot_table-method), a pivot table allows you to aggregate the entries in a DataFrame based on two categorical columns. In this question, you are given a simple dataset, `data/sales.csv`, and are asked to solve a few problems using the `pivot_table` method. 

**We have provided outlines for the DataFrames you need to create, but yours may have a different number of rows and columns and different values.**

***Note***: If it helps, [here](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html) is another great resource that provides an overview of `pivot_table` with many examples from the Titanic dataset.


#### `average_seller`

Complete the implementation of the function `average_seller`, which takes in the `sales` DataFrame and returns a DataFrame that contains the average sales for each seller, indexed by `'Name'` and containing the column `'Average Sales'`. There should not be any `NaN`s.

***Note***: You may be able to implement `average_seller` without using `pivot_table`.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Average Sales</th>
    </tr>
    <tr>
      <th>Name</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Jones</th>
      <td>0</td>
    </tr>
    <tr>
      <th>Smith</th>
      <td>0</td>
    </tr>
    <tr>
      <th>Trump</th>
      <td>0</td>
    </tr>
  </tbody>
</table>

<br>

#### `product_name`

Complete the implementation of the function `product_name` that takes in the `sales` DataFrame and returns a DataFrame that contains the total sales for each product, indexed by `'Name'`. Do not fill in `NaN`s.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th>Product</th>
      <th>boat</th>
      <th>book</th>
      <th>hotel</th>
    </tr>
    <tr>
      <th>Name</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Jones</th>
      <td>NaN</td>
      <td>0.0</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Smith</th>
      <td>NaN</td>
      <td>0.0</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Trump</th>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
    </tr>
  </tbody>
</table>

<br>

#### `count_product`

Complete the implementation of the function `count_product` that takes in the `sales` DataFrame and returns a DataFrame that contains the total number of items sold product-wise and name-wise per date. Replace `NaN`s with 0s. Don't reset the index after pivoting.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th>Date</th>
      <th>01.01.2012</th>
      <th>02.20.2013</th>
      <th>02.25.2015</th>
    </tr>
    <tr>
      <th>Product</th>
      <th>Name</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>boat</th>
      <th>Trump</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">book</th>
      <th>Jones</th>
      <td>0</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>Smith</th>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>Trump</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>hotel</th>
      <th>Trump</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

<br>

#### `total_by_month`

Complete the implementation of the function `total_by_month` that takes in the `sales` DataFrame and returns a pivot table that contains the total sales name-wise, product-wise per month. Replace `NaN`s with 0s. Don't reset the index after pivoting.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th>Month</th>
      <th>February</th>
      <th>January</th>
      <th>July</th>
      <th>March</th>
    </tr>
    <tr>
      <th>Name</th>
      <th>Product</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="3" valign="top">Jones</th>
      <th>book</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>pen</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>ruler</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">Smith</th>
      <th>book</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>pen</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>ruler</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

In [ ]:
# don't change this cell -- it is needed for the tests to work
fp = Path('data') / 'sales.csv'
sales = pd.read_csv(fp)
q4_average_seller_out = average_seller(sales)
q4_product_name_out = product_name(sales)
q4_product_count_out = count_product(sales)
q4_total_by_month_out = total_by_month(sales)

In [ ]:
grader.check("q4")

## Congratulations! You're done Lab 3! 🏁

As a reminder, all of the work you want to submit needs to be in `lab.py`.

To ensure that all of the work you want to submit is in `lab.py`, we've included a script named `lab-validation.py` in the lab folder. You shouldn't edit it, but instead, you should call it from the command line (e.g. the Terminal) to test your work.

Once you've finished the lab, you should open the command line and run, in the directory for this lab:

```
python lab-validation.py
```

**This will run all of the `grader.check` cells that you see in this notebook, but only using the code in `lab.py` – that is, it doesn't look at any of the code in this notebook. If all of your `grader.check` cells pass in this notebook but not all of them pass in your command line with the above command, then you likely have code in your notebook that isn't in your `lab.py`!**

You can also use `lab-validation.py` to test individual questions. For instance,

```
python lab-validation.py q1 q2 q4
```

will run the `grader.check` cells for Questions 1, 2, and 4 – again, only using the code in `lab.py`. [This video](https://www.loom.com/share/0ea254b85b2745e59322b5e5a8692e91?sid=5acc92e6-0dfe-4555-9b6a-8115b6a52f99) how to use the script as well.

Once `python lab-validation.py` shows that you're passing all test cases, you're ready to submit your `lab.py` (and only your `lab.py`) to Gradescope. Once submitting to Gradescope, make sure to stick around until all test cases pass.

There is also a call to `grader.check_all()` below in _this_ notebook, but make sure to also follow the steps above.

<div class="alert alert-block alert-danger" markdown="1">

**There are only two functions in this lab in which you may use a `for`-loop:**
- **`read_linkedin_survey` in Question 1.**
- **`read_student_surveys` in Question 2.**
    
**You may lose points if you use a `for`-loop or `while`-loop in any other question!**

</div>

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()